In [3]:
from datetime import datetime, timedelta
from pandas import DataFrame
from models.Payload import REGION, RegPayload, SecPayload
from models.Source import DataSource as SecApi
from utils.transforms import str2date
from urllib.parse import urlencode as url
#cspell:disable

#input
region:REGION='Valparaiso'
#total regional clients
users = SecApi()
q = users.request('reg_clients',RegPayload(region=region))
clients = int(q[0]['CLIENTES'])
print(f"usuarios regionales: {clients:,.0f} clientes")

#data
curr = SecApi()
curr.request('affected_detail',SecPayload())
curr_df=curr.dataframe(FECHA_INT_STR=str2date)
curr_df= curr_df[curr_df['NOMBRE_REGION']==region].drop(columns=['HORA','DIA','MES','ANHO'])
affected = curr_df['CLIENTES_AFECTADOS'].sum()

print('usuarios sin servicio :',f'{affected:,.0f}',' clientes',f'{affected/clients:.2%}')

res:DataFrame = curr_df.groupby(['NOMBRE_COMUNA','NOMBRE_EMPRESA','FECHA_INT_STR'],as_index=False)['CLIENTES_AFECTADOS'].sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)

#cspell:disable
res:DataFrame = res.rename(columns={
    'FECHA_INT_STR':'INCIDENTE',
    'CLIENTES_AFECTADOS':'SIN_SERVICIO',
    'NOMBRE_EMPRESA':'DISTRIBUIDORA',
    'NOMBRE_COMUNA':'COMUNA'
    })

#days w/o service
res['TIEMPO_SIN_SERVICIO'] = res['INCIDENTE'].apply(lambda event_date: datetime.now()-event_date+timedelta(days=1) )
res

usuarios regionales: 780,563 clientes
usuarios sin servicio : 1,283  clientes 0.16%


,COMUNA,DISTRIBUIDORA,INCIDENTE,SIN_SERVICIO,TIEMPO_SIN_SERVICIO
11,Viña del Mar,CHILQUINTA,2024-02-19,513,183 days 13:43:29.613991
7,San Felipe,CHILQUINTA,2024-08-19,333,1 days 13:43:29.613991
12,Viña del Mar,CHILQUINTA,2024-08-18,222,2 days 13:43:29.613991
0,El Quisco,LITORAL,2024-08-19,109,1 days 13:43:29.613991
5,Quilpue,CHILQUINTA,2024-02-19,83,183 days 13:43:29.613991
8,Santo Domingo,CHILQUINTA,2024-08-19,12,1 days 13:43:29.613991
10,Viña del Mar,CGE,2024-08-19,4,1 days 13:43:29.613991
3,Puchuncavi,CGE,2024-08-19,2,1 days 13:43:29.613991
1,La Ligua,CGE,2024-08-19,1,1 days 13:43:29.613991
2,Papudo,CGE,2024-08-19,1,1 days 13:43:29.613991
